# CHB em csv

Gerando arquivos csv em cada pasta CHB

> `neg` e `pos` em variáveis se referem às classes (`target`):
    
- 0: negativo
- 1: positivo

## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd

from zipfile import ZipFile

from pywt import wavedec
from statsmodels.robust.scale import mad as medianAD

file_range = ['21', '22', '23', '24']

## Carregando funções

In [2]:
features = ['std', 'mean', 'skew', 'kurt', 'meanAD', 'medianAD', 'energy']

# Colunas do Dataset com DWT
decomposition_level = 5

dwt_coefs = [f'A{decomposition_level}'] + [f'D{level}' for level in range(1, decomposition_level + 1)]

labels_com_dwt = [
    f'{feat}-{coef}-{column}' for column in range(18) for coef in dwt_coefs for feat in features
] + ['target']

# Colunas do Dataset sem DWT
labels_sem_dwt = [
    f'{feat}-{column}' for column in range(18) for feat in features
] + ['target']


def energy(mat: np.ndarray) -> np.float64: return (mat ** 2).sum(axis=1)


def extract_features_sem_dwt(matrix: np.ndarray, target: np.float64) -> pd.DataFrame:
    '''
    É esperado como entrada uma matrix de dimensões 18 x 512.

    Durante a execução, um dataframe statsDF de dimensões 7 x 18

    Ao fim um DataFrame com 127 colunas e 1 linha
    126 (7 * 18 do statsDF) + 1 (do target)
    '''

    matrixDF, statsDF = pd.DataFrame(data=matrix).transpose(), pd.DataFrame()

    energy = lambda mat: (mat ** 2).sum(axis=1)

    statsDF['std'] = matrixDF.std()
    statsDF['mean'] = matrixDF.mean()
    statsDF['skew'] = matrixDF.skew()
    statsDF['kurt'] = matrixDF.kurt()
    statsDF['meanAD'] = matrixDF.mad()
    statsDF['medianAD'] = medianAD(matrixDF)
    statsDF['energy'] = energy(matrix)

    # Redimensionando matriz para uma linha e concatenando target à lista
    column = list(statsDF.values.reshape(126)) + [target]

    return pd.DataFrame(data=column, index=labels_sem_dwt).transpose()


def extract_features_com_dwt(matrix: np.ndarray, target: np.float64) -> pd.DataFrame:
    '''
    É esperado como entrada uma matrix de dimensões 18 x 512.

    Durante a execução, para cada uma das 18 colunas,
    é gerado uma lista de 6 arrays de tamanhos variados
    em função da wavedec.
    De cada um dos 6 arrays é extraido 7 features.

    um dataframe statsDF de dimensões 1 x 757 é retornado
    com as features calculadas.
    '''
    statsDF = pd.DataFrame(columns=labels_com_dwt)
    energy = lambda mat: (mat ** 2).sum()

    for i, column in enumerate(matrix):
        wavelet_coefs = wavedec(
            data=column,
            wavelet='db2',
            level=decomposition_level
        )

        for coef, coef_label in zip(wavelet_coefs, dwt_coefs):
            coefDF = pd.DataFrame(data=coef, dtype=np.float64)

            statsDF[f'std-{coef_label}-{i}'] = coefDF.std()
            statsDF[f'mean-{coef_label}-{i}'] = coefDF.mean()
            statsDF[f'skew-{coef_label}-{i}'] = coefDF.skew()
            statsDF[f'kurt-{coef_label}-{i}'] = coefDF.kurt()
            statsDF[f'meanAD-{coef_label}-{i}'] = coefDF.mad()
            statsDF[f'medianAD-{coef_label}-{i}'] = medianAD(coefDF)
            statsDF[f'energy-{coef_label}-{i}'] = energy(coef)

    statsDF['target'] = target
    return statsDF


## Gerando arquivos csv

In [3]:
for i in file_range:
    # Carregando Matrizes de arquivo zip
    with ZipFile(f'./chb{i}/chb{i}.zip') as data:
        # Cria uma lista com os nomes dos arquivos dentro do zip e os ordena
        file_list = data.namelist()
        file_list.sort()

        pos_list = [pos for pos in file_list if (f'chb{i}/positive/' in pos)]
        pos_len = len(pos_list)
        tot_len = len(file_list)
        neg_list = [file_list[i] for i in range(pos_len)]

        print(f'CHB{i}:\tPositive files: {pos_len};\tTotal files: {tot_len};')

        pos_space, neg_space = [], []

        # Cada arquivo é uma matriz que será salva nas listas {pos, neg}_space
        for pos_file, neg_file in zip(pos_list, neg_list):
            with data.open(name=pos_file, mode='r') as pos, data.open(name=neg_file, mode='r') as neg:
                pos_space.append(np.load(pos))
                neg_space.append(np.load(neg))

        # Convertendo listas para arrays
        pos_space = np.array(pos_space, dtype=np.float64)
        neg_space = np.array(neg_space, dtype=np.float64)

        # Gerando dataframe sem DWT
        dataset_sem_dwt = pd.DataFrame(columns=labels_sem_dwt)

        neg_df_list_sem_dwt = [extract_features_sem_dwt(neg_mat, 0) for neg_mat in neg_space]
        pos_df_list_sem_dwt = [extract_features_sem_dwt(pos_mat, 1) for pos_mat in pos_space]

        tot_df_list_sem_dwt = neg_df_list_sem_dwt + pos_df_list_sem_dwt

        dataset_sem_dwt = pd.concat(tot_df_list_sem_dwt, ignore_index=True)

        # Gerando dataframe com DWT
        dataset_com_dwt = pd.DataFrame(columns=labels_com_dwt)

        neg_df_list_com_dwt = [extract_features_com_dwt(neg_mat, 0) for neg_mat in neg_space]
        pos_df_list_com_dwt = [extract_features_com_dwt(pos_mat, 1) for pos_mat in pos_space]

        tot_df_list_com_dwt = neg_df_list_com_dwt + pos_df_list_com_dwt

        dataset_com_dwt = pd.concat(tot_df_list_com_dwt, ignore_index=True)

        # Salvando dataset em arquivo csv
        dataset_sem_dwt.to_csv(path_or_buf=f'./chb{i}/chb{i}_sem_dwt.csv', index=False)
        dataset_com_dwt.to_csv(path_or_buf=f'./chb{i}/chb{i}_com_dwt.csv', index=False)

CHB21:	Positive files: 96;	Total files: 10890;
CHB22:	Positive files: 99;	Total files: 50479;
CHB23:	Positive files: 207;	Total files: 7028;
CHB24:	Positive files: 244;	Total files: 7077;
